In [ ]:
%matplotlib inline

### Utility Functions

## <font color="red">*Pitch Your Project*</font>

<font color="red">In the three cells immediately following, describe **WHAT** you are 
planning to analyze for your final project (i.e., texts, contexts and the social game, 
world and actors you intend to learn about through your analysis) (<200 words), **WHY** 
you are going to do it (i.e., why would theory and/or the average person benefit from 
knowing the results of your investigation) (<200 words), and **HOW** you plan to investigate 
it (i.e., what are the approaches and operations you plan to perform, in sequence, to yield 
this insight) (<400 words).

# ***What? (<200 words)***
For our project, Chanteria Milner and I will analyze congressional and Supreme Court 
abortion-related legislation. Particularly for the congressional legislation, we will explore 
all legislation from 1973 until 2024. We sourced this legislation from the congress.gov 
legislation search, where we filtered for any legislation within this period that could 
have become bills and included the following keywords in the bill text or summary: 
'abortion,' 'reproduction,' or 'reproductive health care.' For the SCOTUS abortion 
legislation, we targeted SCOTUS decisions outlined on supreme.justia.com, which provides a 
list of abortion-relevant SCOTUS decisions from 1965-2022. Through this analysis, we 
plan to uncover the ways that the legislative abortion discourse has changed over time and 
the prominent congressional bills that have occurred throughout subsequent pieces of legislation. 
We will supplement this analysis with legislation in its political history by taking 
note of the political affiliations of congress members, SCOTUS justices, and the presidency.

Chanteria Milner: [Link](https://github.com/chanteriam)
Michael Plunkett: [Link](https://github.com/michplunkett)\
Congressional source: [Link](https://www.congress.gov/advanced-search/legislation?congressGroup%5B%5D=0&congresses%5B%5D=118&congresses%5B%5D=117&congresses%5B%5D=116&congresses%5B%5D=115&congresses%5B%5D=114&congresses%5B%5D=113&congresses%5B%5D=112&congresses%5B%5D=111&congresses%5B%5D=110&congresses%5B%5D=109&congresses%5B%5D=108&congresses%5B%5D=107&congresses%5B%5D=106&congresses%5B%5D=105&congresses%5B%5D=104&congresses%5B%5D=103&congresses%5B%5D=102&congresses%5B%5D=101&congresses%5B%5D=100&congresses%5B%5D=99&congresses%5B%5D=98&congresses%5B%5D=97&congresses%5B%5D=96&congresses%5B%5D=95&congresses%5B%5D=94&congresses%5B%5D=93&legislationNumbers=&restrictionType=field&restrictionFields%5B%5D=allBillTitles&restrictionFields%5B%5D=summary&summaryField=billSummary&enterTerms=%22reproductive+health+care%22%2C+%22reproduction%22%2C+%22abortion%22&legislationTypes%5B%5D=hr&legislationTypes%5B%5D=hjres&legislationTypes%5B%5D=s&legislationTypes%5B%5D=sjres&public=true&private=true&chamber=all&actionTerms=&legislativeActionWordVariants=true&dateOfActionOperator=equal&dateOfActionStartDate=&dateOfActionEndDate=&dateOfActionIsOptions=yesterday&dateOfActionToggle=multi&legislativeAction=Any&sponsorState=One&member=&sponsorTypes%5B%5D=sponsor&sponsorTypeBool=OR&dateOfSponsorshipOperator=equal&dateOfSponsorshipStartDate=&dateOfSponsorshipEndDate=&dateOfSponsorshipIsOptions=yesterday&committeeActivity%5B%5D=0&committeeActivity%5B%5D=3&committeeActivity%5B%5D=11&committeeActivity%5B%5D=12&committeeActivity%5B%5D=4&committeeActivity%5B%5D=2&committeeActivity%5B%5D=5&committeeActivity%5B%5D=9&satellite=null&search=&submitted=Submitted)
SCOTUS source: [Link](https://supreme.justia.com/cases-by-topic/abortion-reproductive-rights/)

## ***Why? (<200 words)***
This analysis, in general, will give us an understanding of how abortion discourse has
changed over time, and specifically, what arguments were used in the passing of the 
1973 Roe v. Wade decision--and with it, assertion of the constitutional right to 
abortion and its eventual reversal in the 2022 Dobbs v. Jackson decision. 
From this, we will be able to uncover what political mechanisms were at play that 
enabled this regression in legislation, and how such significant cases went on to influence 
congressional legislation that followed. The average person will, in general, be able 
to understand how sensitive practices are discussed in the political sphere and what 
aspects in particular are targeted for protection or not. Moreover, this analysis 
will allow individuals who want to write reproductive healthcare legislation to 
understand what arguments are more likely to work over others within certain 
political contexts.

## ***How? (<400 words)***

## <font color="red">*Pitch Your Sample*</font>

<font color="red">In the cell immediately following, describe the rationale behind 
your proposed sample design for your final project. What is the social game, social 
work, or social actors you about whom you are seeking to make inferences? What are 
its virtues with respect to your research questions? What are its limitations? What 
are alternatives? What would be a reasonable path to "scale up" your sample for 
further analysis (i.e., high-profile publication) beyond this class? (<300 words).

## ***Which words? (<300 words)***



## <font color="red">*Exercise 1*</font>

<font color="red">Construct cells immediately below this that embed documents 
related to your final project using at least two different specification of 
`word2vec` and/or `fasttext`, and visualize them each with two separate 
visualization layout specifications (e.g., TSNE, PCA). Then interrogate 
critical word vectors within your corpus in terms of the most similar words, 
analogies, and other additions and subtractions that reveal the structure of 
similarity and difference within your semantic space. What does this pattern 
reveal about the semantic organization of words in your corpora? Which 
estimation and visualization specification generate the most insight and 
appear the most robustly supported and why?

<font color="red">***Stretch***: Explore different vector calculations 
beyond addition and subtraction, such as multiplication, division or some 
other function. What does this exploration reveal about the semantic 
structure of your corpus?

In [ ]:
# Load the

## <font color="red">*Exercise 2*</font>

<font color="red">Construct cells immediately below this that embed 
documents related to your final project using `doc2vec`, and explore 
the relationship between different documents and the word vectors you 
analyzed in the last exercise. Consider the most similar words to 
critical documents, analogies (doc _x_ + word _y_), and other additions 
and subtractions that reveal the structure of similarity and difference 
within your semantic space. What does this pattern reveal about the 
documentary organization of your semantic space?

## <font color="red">*Exercise 3*</font>

<font color="red">Construct cells immediately below this that embed 
documents related to your final project, then generate meaningful 
semantic dimensions based on your theoretical understanding of the 
semantic space (i.e., by subtracting semantically opposite word vectors) 
and project another set of word vectors onto those dimensions. Interpret 
the meaning of these projections for your analysis. Which of the dimensions 
you analyze explain the most variation in the projection of your words and why?

<font color="red">***Stretch***: Average together multiple antonym pairs to 
create robust semantic dimensions. How do word projections on these robust 
dimensions differ from single-pair dimensions?

## <font color="red">*Exercise 4*</font>

<font color="red">Construct cells immediately below this that align 
word embeddings over time or across domains/corpora. Interrogate the 
spaces that result and ask which words changed most and least over the 
entire period or between contexts/corpora. What does this reveal about 
the social game underlying your space?